In [1]:
#数组包
import numpy
#激活函数S函数 expit()
import scipy.special
#绘图包
import matplotlib.pyplot
#在程序内绘图
%matplotlib inline

In [2]:
#神经网络类
class neuralNetwork:
    
    #初始化神经网络
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        #初始化输入层、隐藏层、输出层的结点数
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        #从正态分布曲线提取初始化数值，初始化链接权重（i-input h-hidden o=output）
        self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))
        #初始化学习率lr
        self.lr = learningrate
        #激活函数——S函数
        self.activation_function = lambda x: scipy.special.expit(x)
        pass
    
    #训练神经网络
    def train(self, inputs_list, targets_list):
        #转化数组为二维
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        #计算隐藏层输入
        hidden_inputs = numpy.dot(self.wih, inputs)
        #隐藏层输入->S函数->隐藏层输出
        hidden_outputs = self.activation_function(hidden_inputs)
        #计算输出层输入
        final_inputs = numpy.dot(self.who, hidden_outputs)
        #输出层输入->S函数->输出层输出
        final_outputs = self.activation_function(final_inputs)
        #输出层误差值(target - actual)
        output_errors = targets - final_outputs
        #隐藏层误差值
        hidden_errors = numpy.dot(self.who.T, output_errors) 
        #更新隐藏层和输出层间的链接权重
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        #更新输出入层和隐藏层间的链接权重
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        pass
    
    #测试神经网络
    def query(self, inputs_list):
        #转化数组为二维
        inputs = numpy.array(inputs_list, ndmin=2).T
        #计算隐藏层输入
        hidden_inputs = numpy.dot(self.wih, inputs)
        #隐藏层输入->S函数->隐藏层输出
        hidden_outputs = self.activation_function(hidden_inputs)
        #计算输出层输入
        final_inputs = numpy.dot(self.who, hidden_outputs)
        #输出层输入->S函数->输出层输出
        final_outputs = self.activation_function(final_inputs)
        return final_outputs

In [3]:
#初始化结点
input_nodes = 784
hidden_nodes = 200
output_nodes = 10
#学习率
learning_rate = 0.1
#创建神经网络对象
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

In [15]:
#导入CSV训练数据
training_data_file = open("mnist_train.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [16]:
#训练神经网络
#重复训练 epochs次
epochs = 5
for e in range(epochs):
    #对于训练集的每一条数据
    for record in training_data_list:
        #剔除‘,’
        all_values = record.split(',')
        #将数据值0~255压缩至0.01~0.99
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        #创建期望输出值（如0.01 0.01 0.01 0.99 0.01 0.01 0.01 0.01 0.01 0.01代表数字3）
        targets = numpy.zeros(output_nodes) + 0.01
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        pass
    pass

In [17]:
#导入CSV测试数据
test_data_file = open("mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [18]:
#测试神经网络

#记录神经网络的表现
scorecard = []
#对于测试集中的每一条数据
for record in test_data_list:
    #剔除‘,’
    all_values = record.split(',')
    #期望得到的结果
    correct_label = int(all_values[0])
    #将数据值0~255压缩至0.01~0.99
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    #测试神经网络
    outputs = n.query(inputs)
    #得到测试结果
    label = numpy.argmax(outputs)
    #比照测试结果与期望结果
    if (label == correct_label):
        #识别正确
        scorecard.append(1)
    else:
        #识别错误
        scorecard.append(0)
        pass   
    pass

In [19]:
#计算识别准确度
scorecard_array = numpy.asarray(scorecard)
print ("准确度 = ", scorecard_array.sum() / scorecard_array.size)

准确度 =  0.9731
